Install lizard

It can be ran using in terminal ->lizard fileToAnalizePath

In [1]:
!pip install lizard

In [4]:
import lizard

i = lizard.analyze_file("/Users/lukas./Desktop/CMPUT660Project/test.py")

In [17]:
import lizard
import subprocess

# Path to the file
file_path = "/Users/lukas./Desktop/CMPUT660Project/test.py"

# Analyze the file (this returns FileInformation)
subprocess.run([f"lizard", file_path])


  NLOC    CCN   token  PARAM  length  location  
------------------------------------------------
       4      2     22      0       5 test@2-6@/Users/lukas./Desktop/CMPUT660Project/test.py
       2      1      8      0       3 test2@8-10@/Users/lukas./Desktop/CMPUT660Project/test.py
1 file analyzed.
NLOC    Avg.NLOC  AvgCCN  Avg.token  function_cnt    file
--------------------------------------------------------------
      7       3.0     1.5       15.0         2     /Users/lukas./Desktop/CMPUT660Project/test.py

No thresholds exceeded (cyclomatic_complexity > 15 or length > 1000 or nloc > 1000000 or parameter_count > 100)
Total nloc   Avg.NLOC  AvgCCN  Avg.token   Fun Cnt  Warning cnt   Fun Rt   nloc Rt
------------------------------------------------------------------------------------------
         7       3.0     1.5       15.0        2            0      0.00    0.00


CompletedProcess(args=['lizard', '/Users/lukas./Desktop/CMPUT660Project/test.py'], returncode=0)

The following cell will analyze a single file


In [ ]:
import subprocess
import re
import os

folder_path = "/Users/lukas./Desktop/CMPUT660Project/code_complexity/test_fold"
results = []

files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

all_metrics = []
max_values = {
    "max_nloc": 0,
    "max_ccn": 0,
    "max_tokens": 0,
    "max_params": 0,
    "max_length": 0
}
for file_path in files:
    result = subprocess.run(
        ["lizard", file_path],
        capture_output=True,
        text=True
    )
    cli_output = result.stdout

    # per function metrics
    function_pattern = r"\s*(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(.+)"
    functions_info = []
    
    for line in cli_output.splitlines():
        match = re.match(function_pattern, line)
        if match:
            nloc, ccn, tokens, params, length, location = match.groups()
            functions_info.append({
                "nloc": int(nloc),
                "ccn": int(ccn),
                "tokens": int(tokens),
                "params": int(params),
                "length": int(length),
                "location": location.strip()
            })
            if int(nloc) > max_values["max_nloc"]:
                max_values["max_nloc"] = int(nloc)
            if int(ccn) > max_values["max_ccn"]:
                max_values["max_ccn"] = int(ccn)
            if int(tokens) > max_values["max_tokens"]:
                max_values["max_tokens"] = int(tokens)
            if int(params) > max_values["max_params"]:
                max_values["max_params"] = int(params)
            if int(length) > max_values["max_length"]:
                max_values["max_length"] = int(length)
    # file level summary
    file_summary_pattern = r"^\s*(\d+)\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)\s+(\d+)\s+(.+)$"
    file_summary = None
    for line in cli_output.splitlines():
        match = re.match(file_summary_pattern, line)
        if match:
            nloc, avg_nloc, avg_ccn, avg_tokens, function_count, filename = match.groups()
            if not any(c.isalpha() for c in filename):  # skip header lines
                file_summary = {
                    "total_nloc": int(nloc),
                    "avg_nloc": float(avg_nloc),
                    "avg_ccn": float(avg_ccn),
                    "avg_tokens": float(avg_tokens),
                    "function_count": int(function_count),
                    "filename": filename.strip()
                }
                break

    all_metrics.append({
        "file_path": file_path,
        "functions_info": functions_info,
        "file_summary": file_summary,
        "raw_output": cli_output
    })


for metric in all_metrics:
    print(f"\nFile: {metric['file_path']}")
    print("Functions info:")
    for f in metric["functions_info"]:
        print(f)
    print("File summary:")
    print(metric["file_summary"])

print("MaxValues across all files:")
print(max_values)


File: /Users/lukas./Desktop/CMPUT660Project/code_complexity/test_fold/test.py
Functions info:
{'nloc': 4, 'ccn': 2, 'tokens': 22, 'params': 0, 'length': 5, 'location': 'test@2-6@/Users/lukas./Desktop/CMPUT660Project/code_complexity/test_fold/test.py'}
{'nloc': 2, 'ccn': 1, 'tokens': 8, 'params': 0, 'length': 3, 'location': 'test2@8-10@/Users/lukas./Desktop/CMPUT660Project/code_complexity/test_fold/test.py'}
File summary:
{'total_nloc': 7, 'avg_nloc': 3.0, 'avg_ccn': 1.5, 'avg_tokens': 15.0, 'function_count': 2, 'filename': '0      0.00    0.00'}

File: /Users/lukas./Desktop/CMPUT660Project/code_complexity/test_fold/test2.c
Functions info:
{'nloc': 3, 'ccn': 1, 'tokens': 15, 'params': 2, 'length': 3, 'location': 'add@4-6@/Users/lukas./Desktop/CMPUT660Project/code_complexity/test_fold/test2.c'}
{'nloc': 7, 'ccn': 2, 'tokens': 31, 'params': 1, 'length': 7, 'location': 'factorial@9-15@/Users/lukas./Desktop/CMPUT660Project/code_complexity/test_fold/test2.c'}
{'nloc': 9, 'ccn': 4, 'tokens':